In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2021/SampleSubmissionWiDS2021.csv
/kaggle/input/widsdatathon2021/SolutionTemplateWiDS2021.csv
/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv
/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv
/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv


In [79]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn import metrics
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Classifiers
from sklearn.svm import NuSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier # <- Here is our boy

from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [80]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import mean_squared_error, classification_report
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

# import xgboost
# import lightgbm as lgb
# from lightgbm import LGBMClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
pd.set_option('max_rows', 300)
import re

In [81]:
pd.set_option('display.max_columns', 300)
np.random.seed(566)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', -1)

In [82]:
df1 =pd.read_csv('TrainingWiDS2021.csv')
df2 =pd.read_csv('UnlabeledWiDS2021.csv')

In [83]:
y=df1['diabetes_mellitus']

In [84]:
df = df1.drop(['diabetes_mellitus'], axis=1).append(df2)
df.shape

(140391, 180)

In [85]:
df.drop(df.columns[df.columns.str.contains('^h1')], axis=1, inplace=True)
df.drop(['hospital_id'], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('^Un')], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('invasive')], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('noninvasive')], axis=1, inplace=True)
df.drop(['icu_id'], axis=1, inplace=True)
cols = df.columns[df.isnull().mean()>0.7]
print(cols)

Index(['fio2_apache', 'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache', 'ph_apache', 'd1_lactate_max', 'd1_lactate_min', 'd1_pao2fio2ratio_max', 'd1_pao2fio2ratio_min'], dtype='object')


In [86]:
df.drop(cols, axis=1, inplace=True)
df.drop(['readmission_status'], axis=1,inplace=True)
df.drop(['hepatic_failure'], axis=1,inplace=True)
threshold = 0.0
df.drop(df.std()[df.std()==threshold].index.values, axis=1,inplace=True)
df.shape

(140391, 90)

In [87]:
df['pp']=df['d1_sysbp_max']-df['d1_diasbp_max']
df['gcs']=df['gcs_eyes_apache']+df['gcs_motor_apache']+df['gcs_verbal_apache']
df['bmi_calc'] = df['weight']/(df['height']/100)**2
df.drop(['bmi'],axis=1,inplace=True)
df.shape

(140391, 92)

In [88]:
df['gcs']=df['gcs']+df['gcs_unable_apache']
df.drop(['gcs_eyes_apache'],axis=1,inplace=True)
df.drop(['gcs_motor_apache'],axis=1,inplace=True)
df.drop(['gcs_verbal_apache'],axis=1,inplace=True)
df.drop(['gcs_unable_apache'],axis=1,inplace=True)
df.shape

(140391, 88)

In [89]:
df.shape

(140391, 88)

In [90]:
df.head(3)

,encounter_id,age,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_temp_max,d1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,aids,cirrhosis,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,pp,gcs,bmi_calc
0,214826,68.00,0,Caucasian,M,180.30,Floor,Floor,admit,CTICU,0.54,73.90,2.30,113.00,502.01,0,0,0.40,31.00,2.51,168.00,118.00,27.40,0,40.00,36.00,134.00,39.30,nan,0,14.10,68.00,37.00,119.00,72.00,89.00,46.00,34.00,10.00,100.00,74.00,131.00,73.00,39.90,37.20,2.30,2.30,0.40,0.40,31.00,30.00,8.50,7.40,2.51,2.23,168.00,109.00,19.00,15.00,8.90,8.90,27.40,27.40,nan,nan,233.00,233.00,4.00,3.40,136.00,134.00,14.10,14.10,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,63.00,13.00,22.73
1,246060,77.00,0,Caucasian,F,160.00,Floor,Floor,admit,Med-Surg ICU,0.93,70.20,nan,108.00,203.01,0,0,nan,9.00,0.56,145.00,120.00,36.90,0,46.00,33.00,145.00,35.10,nan,1,12.70,95.00,31.00,118.00,72.00,120.00,38.00,32.00,12.00,100.00,70.00,159.00,67.00,36.30,35.10,1.60,1.60,0.50,0.50,11.00,9.00,8.60,8.00,0.71,0.56,145.00,128.00,27.00,26.00,11.30,11.10,36.90,36.10,1.30,1.30,557.00,487.00,4.20,3.80,145.00,145.00,23.30,12.70,37.00,37.00,7.45,7.45,51.00,51.00,0,0,0,0,0,0,64.00,5.00,27.42
2,276985,25.00,0,Caucasian,F,172.70,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.00,95.30,nan,122.00,703.03,0,0,nan,nan,nan,nan,102.00,nan,0,68.00,37.00,nan,36.70,nan,0,nan,88.00,48.00,96.00,68.00,102.00,68.00,21.00,8.00,98.00,91.00,148.00,105.00,37.00,36.70,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,60.00,14.00,31.95


In [91]:
existing_cat_columns=[]
for col in df.columns:
    if df[col].dtype =='object':
        existing_cat_columns.append(col)
		
print(existing_cat_columns)

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']


In [92]:
from sklearn.preprocessing import LabelEncoder
for c in df.columns:
 if df[c].dtype == 'object':
     lbl = LabelEncoder()
     lbl.fit(list(df[c].values) + list(df[c].values))
     df[c] = lbl.transform(list(df[c].values))

In [93]:
df[existing_cat_columns].isnull().sum(axis=0)

ethnicity                0
gender                   0
hospital_admit_source    0
icu_admit_source         0
icu_stay_type            0
icu_type                 0
dtype: int64

In [94]:
binaryColumns = ['elective_surgery','apache_post_operative','intubated_apache','ventilated_apache','aids','cirrhosis','immunosuppression','leukemia','lymphoma','solid_tumor_with_metastasis']

In [95]:
for col in binaryColumns:
    df[col].fillna(df[col].mode()[0],inplace=True)

In [96]:
df[binaryColumns].isnull().sum(axis=0)

elective_surgery               0
apache_post_operative          0
intubated_apache               0
ventilated_apache              0
aids                           0
cirrhosis                      0
immunosuppression              0
leukemia                       0
lymphoma                       0
solid_tumor_with_metastasis    0
dtype: int64

In [97]:
existing_numerical_columns=[]
for col in df.columns:
    if df[col].dtype =='int32' or df[col].dtype =='float32' or df[col].dtype =='int64' or df[col].dtype =='float64':
        existing_numerical_columns.append(col)
for i in existing_numerical_columns:
    print(df[i].value_counts())

266238    1
167163    1
269609    1
267560    1
175359    1
         ..
141840    1
145938    1
143891    1
137750    1
262144    1
Name: encounter_id, Length: 140391, dtype: int64
67.00    3447
68.00    3293
71.00    3291
66.00    3217
72.00    3201
65.00    3146
63.00    3104
70.00    3096
73.00    3002
62.00    2980
69.00    2953
64.00    2933
74.00    2931
75.00    2907
61.00    2894
59.00    2869
60.00    2869
77.00    2836
76.00    2824
57.00    2796
78.00    2755
58.00    2711
79.00    2669
56.00    2666
55.00    2637
54.00    2531
80.00    2485
81.00    2464
53.00    2429
82.00    2429
83.00    2365
84.00    2328
52.00    2298
51.00    2119
85.00    2107
86.00    1985
50.00    1982
87.00    1859
49.00    1758
88.00    1623
48.00    1545
47.00    1489
89.00    1431
46.00    1425
45.00    1382
44.00    1187
43.00    1152
42.00    1106
41.00    991 
36.00    903 
40.00    899 
39.00    852 
38.00    834 
34.00    824 
35.00    820 
37.00    816 
33.00    760 
30.00    754 
32.00  

3.10    3621
3.00    3602
2.90    3560
2.80    3448
3.20    3448
3.30    3272
2.70    3201
3.40    3096
2.60    3015
3.50    2810
2.50    2783
2.40    2614
2.30    2440
3.60    2409
2.20    2175
3.70    2056
2.10    1907
2.00    1693
3.80    1667
1.90    1367
3.90    1353
1.80    1074
4.00    1052
1.70    910 
1.60    773 
4.10    765 
1.50    559 
4.20    524 
1.10    489 
1.40    391 
4.30    383 
1.30    271 
4.50    267 
4.40    234 
1.20    213 
3.35    1   
3.44    1   
2.96    1   
Name: d1_albumin_min, dtype: int64
 0.40    6948
 0.50    6573
 0.30    5795
 0.60    5566
 0.70    4610
         ... 
13.90    1   
31.10    1   
40.40    1   
 5.88    1   
31.60    1   
Name: d1_bilirubin_max, Length: 602, dtype: int64
 0.40    7448
 0.50    6732
 0.30    6357
 0.60    5624
 0.70    4491
         ... 
12.50    1   
 2.55    1   
36.50    1   
 2.28    1   
30.07    1   
Name: d1_bilirubin_min, Length: 566, dtype: int64
13.00    5618
14.00    5576
12.00    5529
15.00    5413
11.00  

139.00    13022
140.00    12852
138.00    12251
141.00    11461
137.00    10372
142.00    9500 
136.00    8355 
143.00    7000 
135.00    6208 
144.00    5272 
134.00    4695 
145.00    3709 
133.00    3196 
146.00    2545 
132.00    2331 
131.00    1779 
147.00    1740 
148.00    1178 
130.00    1176 
149.00    873  
129.00    870  
123.00    862  
150.00    654  
128.00    624  
158.00    570  
127.00    469  
151.00    435  
126.00    361  
152.00    342  
153.00    266  
125.00    247  
124.00    221  
154.00    184  
155.00    169  
156.00    119  
157.00    101  
140.20    3    
137.80    2    
138.10    2    
140.50    2    
141.60    2    
137.70    2    
134.40    2    
138.30    2    
139.70    2    
144.20    1    
147.10    1    
145.80    1    
131.40    1    
151.10    1    
132.60    1    
146.20    1    
135.40    1    
145.90    1    
135.60    1    
145.30    1    
139.90    1    
139.50    1    
141.40    1    
140.60    1    
142.50    1    
139.10    1    
133.20  

In [98]:
df[existing_numerical_columns].isnull().sum(axis=0)

encounter_id                   0    
age                            4988 
elective_surgery               0    
ethnicity                      0    
gender                         0    
height                         2378 
hospital_admit_source          0    
icu_admit_source               0    
icu_stay_type                  0    
icu_type                       0    
pre_icu_los_days               0    
weight                         4371 
albumin_apache                 84218
apache_2_diagnosis             1855 
apache_3j_diagnosis            973  
apache_post_operative          0    
arf_apache                     0    
bilirubin_apache               88870
bun_apache                     27472
creatinine_apache              26925
glucose_apache                 15906
heart_rate_apache              339  
hematocrit_apache              28872
intubated_apache               0    
map_apache                     472  
resprate_apache                872  
sodium_apache                  26512
t

In [99]:
nonBinaryColumns = [x for x in existing_numerical_columns if x not in binaryColumns]
print(nonBinaryColumns)

['encounter_id', 'age', 'ethnicity', 'gender', 'height', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'pre_icu_los_days', 'weight', 'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache', 'bilirubin_apache', 'bun_apache', 'creatinine_apache', 'glucose_apache', 'heart_rate_apache', 'hematocrit_apache', 'map_apache', 'resprate_apache', 'sodium_apache', 'temp_apache', 'urineoutput_apache', 'wbc_apache', 'd1_diasbp_max', 'd1_diasbp_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 'd1_temp_max', 'd1_temp_min', 'd1_albumin_max', 'd1_albumin_min', 'd1_bilirubin_max', 'd1_bilirubin_min', 'd1_bun_max', 'd1_bun_min', 'd1_calcium_max', 'd1_calcium_min', 'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_glucose_min', 'd1_hco3_max', 'd1_hco3_min', 'd1_hemaglobin_max', 'd1_hemaglobin_min', 'd1_hematocrit_max', 'd1

In [100]:
for i in nonBinaryColumns:
    df[i].fillna(df[i].mean(),inplace=True)

In [101]:
df[nonBinaryColumns].isnull().sum(axis=0)

encounter_id             0
age                      0
ethnicity                0
gender                   0
height                   0
hospital_admit_source    0
icu_admit_source         0
icu_stay_type            0
icu_type                 0
pre_icu_los_days         0
weight                   0
albumin_apache           0
apache_2_diagnosis       0
apache_3j_diagnosis      0
arf_apache               0
bilirubin_apache         0
bun_apache               0
creatinine_apache        0
glucose_apache           0
heart_rate_apache        0
hematocrit_apache        0
map_apache               0
resprate_apache          0
sodium_apache            0
temp_apache              0
urineoutput_apache       0
wbc_apache               0
d1_diasbp_max            0
d1_diasbp_min            0
d1_heartrate_max         0
d1_heartrate_min         0
d1_mbp_max               0
d1_mbp_min               0
d1_resprate_max          0
d1_resprate_min          0
d1_spo2_max              0
d1_spo2_min              0
d

In [102]:
df.shape

(140391, 88)

In [103]:
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 10
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [104]:
final_iv, IV  = data_vars(df[:len(df1)],y)

In [105]:
# Sorting the column in Descending order 
IV = IV.sort_values('IV',ascending=False)

In [106]:
iv_columns = IV.loc[(IV['IV'] > 0.020) ,'VAR_NAME']

In [107]:
len(iv_columns)

21

In [108]:
df_woe = df[iv_columns]

In [109]:
X = df_woe[:len(df1)]
XPred = df_woe[len(df1):].reset_index(drop=True)

In [110]:
#X = df[:len(df1)]
#XPred = df[len(df1):].reset_index(drop=True)

In [112]:
###############################################################################
#                        3. Create train and test set                         #
###############################################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50,random_state = 1000, stratify=y)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train= scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [113]:
###############################################################################
#                              5. Good Ol' Classifiers                        #
###############################################################################
# Initializing Support Vector classifier
classifier1 = SVC(C = 50, degree = 1, gamma = "auto", kernel = "rbf", probability = True)
# Initializing Multi-layer perceptron  classifier
#classifier2 = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
#                            learning_rate = "constant", max_iter = 2000, random_state = 1000)

# Initialing Nu Support Vector classifier
#classifier3 = NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)

# Initializing Random Forest classifier
classifier4 = RandomForestClassifier(n_estimators = 500, criterion = "gini", max_depth = 10,
                                     max_features = "auto", min_samples_leaf = 0.005,
                                     min_samples_split = 0.005, n_jobs = -1, random_state = 1000)

In [114]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

layer_one_estimators = [
                        ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5))             
                       ]
layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier()),
                        ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())

# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,stratify=y, random_state=42)
clf.fit(X_train, y_train).score(X_test, y_test)

0.8038384117764563

In [115]:
ypred=clf.predict(XPred)

In [30]:
ypred_prob=clf.predict_proba(XPred)

In [31]:
pd.DataFrame({'encounter_id':df2['encounter_id'].values,
    'diabetes_mellitus':clf.predict_proba(XPred)[:,1]}).to_csv('Predictions_stacking_now.csv',
                                                                   index=False)

In [32]:
len(ypred_prob)

10234

In [85]:
ypred_prob[0]

array([0.85637927, 0.14362073])

In [76]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import mean_squared_error, classification_report
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

In [77]:
layer_one_estimators = [
                        ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5))             
                       ]
layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier()),
                        ('cb2_classifier',CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15))
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())

# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
clf.fit(X_train, y_train).score(X_test, y_test)

0:	total: 36.7ms	remaining: 18.3s
1:	total: 64ms	remaining: 15.9s
2:	total: 86.1ms	remaining: 14.3s
3:	total: 110ms	remaining: 13.6s
4:	total: 130ms	remaining: 12.9s
5:	total: 151ms	remaining: 12.5s
6:	total: 170ms	remaining: 12s
7:	total: 190ms	remaining: 11.7s
8:	total: 217ms	remaining: 11.8s
9:	total: 238ms	remaining: 11.7s
10:	total: 300ms	remaining: 13.3s
11:	total: 321ms	remaining: 13s
12:	total: 372ms	remaining: 13.9s
13:	total: 392ms	remaining: 13.6s
14:	total: 412ms	remaining: 13.3s
15:	total: 458ms	remaining: 13.9s
16:	total: 481ms	remaining: 13.7s
17:	total: 512ms	remaining: 13.7s
18:	total: 552ms	remaining: 14s
19:	total: 569ms	remaining: 13.7s
20:	total: 615ms	remaining: 14s
21:	total: 639ms	remaining: 13.9s
22:	total: 662ms	remaining: 13.7s
23:	total: 691ms	remaining: 13.7s
24:	total: 710ms	remaining: 13.5s
25:	total: 769ms	remaining: 14s
26:	total: 793ms	remaining: 13.9s
27:	total: 827ms	remaining: 13.9s
28:	total: 857ms	remaining: 13.9s
29:	total: 878ms	remaining: 13.8s

240:	total: 7.04s	remaining: 7.57s
241:	total: 7.07s	remaining: 7.53s
242:	total: 7.08s	remaining: 7.49s
243:	total: 7.11s	remaining: 7.46s
244:	total: 7.15s	remaining: 7.44s
245:	total: 7.17s	remaining: 7.41s
246:	total: 7.2s	remaining: 7.38s
247:	total: 7.24s	remaining: 7.36s
248:	total: 7.27s	remaining: 7.33s
249:	total: 7.29s	remaining: 7.29s
250:	total: 7.32s	remaining: 7.26s
251:	total: 7.36s	remaining: 7.24s
252:	total: 7.41s	remaining: 7.24s
253:	total: 7.44s	remaining: 7.21s
254:	total: 7.47s	remaining: 7.18s
255:	total: 7.5s	remaining: 7.15s
256:	total: 7.53s	remaining: 7.12s
257:	total: 7.56s	remaining: 7.09s
258:	total: 7.58s	remaining: 7.05s
259:	total: 7.6s	remaining: 7.02s
260:	total: 7.63s	remaining: 6.99s
261:	total: 7.66s	remaining: 6.96s
262:	total: 7.69s	remaining: 6.93s
263:	total: 7.71s	remaining: 6.89s
264:	total: 7.74s	remaining: 6.86s
265:	total: 7.76s	remaining: 6.83s
266:	total: 7.79s	remaining: 6.8s
267:	total: 7.82s	remaining: 6.77s
268:	total: 7.87s	remain

482:	total: 14.3s	remaining: 502ms
483:	total: 14.3s	remaining: 472ms
484:	total: 14.3s	remaining: 442ms
485:	total: 14.4s	remaining: 413ms
486:	total: 14.4s	remaining: 384ms
487:	total: 14.4s	remaining: 354ms
488:	total: 14.4s	remaining: 325ms
489:	total: 14.5s	remaining: 296ms
490:	total: 14.5s	remaining: 266ms
491:	total: 14.5s	remaining: 236ms
492:	total: 14.6s	remaining: 207ms
493:	total: 14.6s	remaining: 177ms
494:	total: 14.6s	remaining: 148ms
495:	total: 14.7s	remaining: 118ms
496:	total: 14.7s	remaining: 88.6ms
497:	total: 14.7s	remaining: 59ms
498:	total: 14.7s	remaining: 29.5ms
499:	total: 14.8s	remaining: 0us
0:	total: 62.5ms	remaining: 31.2s
1:	total: 80ms	remaining: 19.9s
2:	total: 99.7ms	remaining: 16.5s
3:	total: 121ms	remaining: 15s
4:	total: 142ms	remaining: 14s
5:	total: 194ms	remaining: 16s
6:	total: 223ms	remaining: 15.7s
7:	total: 264ms	remaining: 16.3s
8:	total: 330ms	remaining: 18s
9:	total: 347ms	remaining: 17s
10:	total: 364ms	remaining: 16.2s
11:	total: 390ms

229:	total: 5.58s	remaining: 6.55s
230:	total: 5.6s	remaining: 6.52s
231:	total: 5.62s	remaining: 6.5s
232:	total: 5.64s	remaining: 6.47s
233:	total: 5.66s	remaining: 6.44s
234:	total: 5.68s	remaining: 6.41s
235:	total: 5.7s	remaining: 6.38s
236:	total: 5.72s	remaining: 6.34s
237:	total: 5.74s	remaining: 6.32s
238:	total: 5.76s	remaining: 6.29s
239:	total: 5.79s	remaining: 6.27s
240:	total: 5.81s	remaining: 6.25s
241:	total: 5.83s	remaining: 6.21s
242:	total: 5.86s	remaining: 6.2s
243:	total: 5.87s	remaining: 6.16s
244:	total: 5.89s	remaining: 6.13s
245:	total: 5.91s	remaining: 6.1s
246:	total: 5.94s	remaining: 6.09s
247:	total: 5.96s	remaining: 6.06s
248:	total: 5.99s	remaining: 6.03s
249:	total: 6s	remaining: 6s
250:	total: 6.03s	remaining: 5.98s
251:	total: 6.05s	remaining: 5.96s
252:	total: 6.07s	remaining: 5.93s
253:	total: 6.09s	remaining: 5.9s
254:	total: 6.12s	remaining: 5.88s
255:	total: 6.15s	remaining: 5.86s
256:	total: 6.18s	remaining: 5.84s
257:	total: 6.2s	remaining: 5.81

471:	total: 10.9s	remaining: 646ms
472:	total: 10.9s	remaining: 623ms
473:	total: 10.9s	remaining: 601ms
474:	total: 11s	remaining: 577ms
475:	total: 11s	remaining: 554ms
476:	total: 11s	remaining: 531ms
477:	total: 11s	remaining: 507ms
478:	total: 11s	remaining: 484ms
479:	total: 11.1s	remaining: 461ms
480:	total: 11.1s	remaining: 437ms
481:	total: 11.1s	remaining: 414ms
482:	total: 11.1s	remaining: 391ms
483:	total: 11.1s	remaining: 368ms
484:	total: 11.2s	remaining: 345ms
485:	total: 11.2s	remaining: 322ms
486:	total: 11.2s	remaining: 299ms
487:	total: 11.2s	remaining: 276ms
488:	total: 11.3s	remaining: 253ms
489:	total: 11.3s	remaining: 230ms
490:	total: 11.3s	remaining: 207ms
491:	total: 11.3s	remaining: 184ms
492:	total: 11.3s	remaining: 161ms
493:	total: 11.4s	remaining: 138ms
494:	total: 11.4s	remaining: 115ms
495:	total: 11.4s	remaining: 92ms
496:	total: 11.4s	remaining: 69ms
497:	total: 11.5s	remaining: 46ms
498:	total: 11.5s	remaining: 23ms
499:	total: 11.5s	remaining: 0us
0

...

177:	total: 5.86s	remaining: 10.6s
178:	total: 5.89s	remaining: 10.6s
179:	total: 5.93s	remaining: 10.5s
180:	total: 5.95s	remaining: 10.5s
181:	total: 5.98s	remaining: 10.4s
182:	total: 6.01s	remaining: 10.4s
183:	total: 6.04s	remaining: 10.4s
184:	total: 6.1s	remaining: 10.4s
185:	total: 6.13s	remaining: 10.3s
186:	total: 6.16s	remaining: 10.3s
187:	total: 6.21s	remaining: 10.3s
188:	total: 6.26s	remaining: 10.3s
189:	total: 6.3s	remaining: 10.3s
190:	total: 6.33s	remaining: 10.2s
191:	total: 6.36s	remaining: 10.2s
192:	total: 6.38s	remaining: 10.1s
193:	total: 6.4s	remaining: 10.1s
194:	total: 6.42s	remaining: 10s
195:	total: 6.45s	remaining: 10s
196:	total: 6.48s	remaining: 9.97s
197:	total: 6.51s	remaining: 9.94s
198:	total: 6.55s	remaining: 9.91s
199:	total: 6.57s	remaining: 9.86s
200:	total: 6.6s	remaining: 9.82s
201:	total: 6.64s	remaining: 9.79s
202:	total: 6.68s	remaining: 9.77s
203:	total: 6.71s	remaining: 9.74s
204:	total: 6.72s	remaining: 9.68s
205:	total: 6.76s	remaining:

413:	total: 12.8s	remaining: 2.67s
414:	total: 12.9s	remaining: 2.64s
415:	total: 12.9s	remaining: 2.61s
416:	total: 12.9s	remaining: 2.58s
417:	total: 13s	remaining: 2.54s
418:	total: 13s	remaining: 2.51s
419:	total: 13s	remaining: 2.48s
420:	total: 13.1s	remaining: 2.45s
421:	total: 13.1s	remaining: 2.42s
422:	total: 13.2s	remaining: 2.39s
423:	total: 13.2s	remaining: 2.36s
424:	total: 13.2s	remaining: 2.33s
425:	total: 13.2s	remaining: 2.3s
426:	total: 13.3s	remaining: 2.27s
427:	total: 13.3s	remaining: 2.24s
428:	total: 13.3s	remaining: 2.21s
429:	total: 13.4s	remaining: 2.17s
430:	total: 13.4s	remaining: 2.15s
431:	total: 13.4s	remaining: 2.12s
432:	total: 13.5s	remaining: 2.09s
433:	total: 13.5s	remaining: 2.06s
434:	total: 13.5s	remaining: 2.02s
435:	total: 13.6s	remaining: 1.99s
436:	total: 13.6s	remaining: 1.96s
437:	total: 13.6s	remaining: 1.93s
438:	total: 13.7s	remaining: 1.9s
439:	total: 13.7s	remaining: 1.87s
440:	total: 13.7s	remaining: 1.84s
441:	total: 13.8s	remaining:

154:	total: 5.08s	remaining: 11.3s
155:	total: 5.13s	remaining: 11.3s
156:	total: 5.18s	remaining: 11.3s
157:	total: 5.24s	remaining: 11.3s
158:	total: 5.29s	remaining: 11.4s
159:	total: 5.33s	remaining: 11.3s
160:	total: 5.37s	remaining: 11.3s
161:	total: 5.4s	remaining: 11.3s
162:	total: 5.43s	remaining: 11.2s
163:	total: 5.46s	remaining: 11.2s
164:	total: 5.49s	remaining: 11.1s
165:	total: 5.51s	remaining: 11.1s
166:	total: 5.54s	remaining: 11s
167:	total: 5.57s	remaining: 11s
168:	total: 5.6s	remaining: 11s
169:	total: 5.63s	remaining: 10.9s
170:	total: 5.66s	remaining: 10.9s
171:	total: 5.68s	remaining: 10.8s
172:	total: 5.72s	remaining: 10.8s
173:	total: 5.77s	remaining: 10.8s
174:	total: 5.82s	remaining: 10.8s
175:	total: 5.85s	remaining: 10.8s
176:	total: 5.89s	remaining: 10.8s
177:	total: 5.93s	remaining: 10.7s
178:	total: 5.97s	remaining: 10.7s
179:	total: 6s	remaining: 10.7s
180:	total: 6.05s	remaining: 10.7s
181:	total: 6.08s	remaining: 10.6s
182:	total: 6.12s	remaining: 10

393:	total: 14.3s	remaining: 3.84s
394:	total: 14.3s	remaining: 3.81s
395:	total: 14.3s	remaining: 3.77s
396:	total: 14.4s	remaining: 3.73s
397:	total: 14.4s	remaining: 3.69s
398:	total: 14.4s	remaining: 3.65s
399:	total: 14.5s	remaining: 3.62s
400:	total: 14.5s	remaining: 3.58s
401:	total: 14.5s	remaining: 3.54s
402:	total: 14.6s	remaining: 3.5s
403:	total: 14.6s	remaining: 3.46s
404:	total: 14.6s	remaining: 3.43s
405:	total: 14.6s	remaining: 3.39s
406:	total: 14.7s	remaining: 3.35s
407:	total: 14.7s	remaining: 3.31s
408:	total: 14.7s	remaining: 3.28s
409:	total: 14.8s	remaining: 3.24s
410:	total: 14.8s	remaining: 3.2s
411:	total: 14.8s	remaining: 3.17s
412:	total: 14.9s	remaining: 3.13s
413:	total: 14.9s	remaining: 3.09s
414:	total: 14.9s	remaining: 3.06s
415:	total: 15s	remaining: 3.02s
416:	total: 15s	remaining: 2.99s
417:	total: 15s	remaining: 2.95s
418:	total: 15.1s	remaining: 2.91s
419:	total: 15.1s	remaining: 2.88s
420:	total: 15.1s	remaining: 2.84s
421:	total: 15.2s	remaining:

0.8038384117764563

In [78]:
ypred_prob=clf.predict_proba(XPred)

In [92]:
pd.DataFrame({'encounter_id':df2['encounter_id'].values,
    'diabetes_mellitus':clf.predict_proba(XPred)[:,1]}).to_csv('Predictions_stackingnow_Cat.csv',
                                                                   index=False)

In [93]:
# anything below not to be considered

# Anything below not to be considered

In [83]:
import pandas
df_csv = pandas.DataFrame(data={"col1": df2['encounter_id'], "col2": ypred_prob})
#df_csv.to_csv("./stacking.csv", sep=',',index=False)


ValueError: Data must be 1-dimensional

In [ ]:
df_csv[['col1','col2']].to_csv("submission_stacking.csv",index=False)

In [ ]:
df_csv.head(3)

In [ ]:
df_csv.nunique()

In [ ]:
###############################################################################
#                             6. Stacking Classifier                          #
###############################################################################
# Initializing the StackingCV classifier
sclf = StackingClassifier(classifiers = [classifier1, classifier4],
                            shuffle = False,
                            use_probas = True,
                            cv = 5,
                            meta_classifier = SVC(probability = True))

In [ ]:
###############################################################################
#                       7. Putting classifiers in a dictionary                #
###############################################################################
# Create list to store classifiers
classifiers = {"SVC": classifier1,"MLP": classifier2,"NuSVC": classifier3,"RF": classifier4,"Stack": sclf}

classifiers = {"SVC": classifier1,RF": classifier4,"Stack": sclf}

In [ ]:
###############################################################################
#                               8. Train classifiers                          #
###############################################################################
# Train classifiers
for key in classifiers:
    # Get classifier
    classifier = classifiers[key]
    
    # Fit classifier
    classifier.fit(X_train, y_train)
        
    # Save fitted classifier
    classifiers[key] = classifier			   

In [ ]:
###############################################################################
#                              9. Making predictions                          #
###############################################################################
# Get results
results = pd.DataFrame()
for key in classifiers:
    # Make prediction on test set
    y_pred = classifiers[key].predict_proba(X_test)[:,1]
    
    # Save results in pandas dataframe object
    results[f"{key}"] = y_pred

# Add the test set to the results object
results["Target"] = y_test

In [ ]:
TARGET_COL = "diabetes_mellitus"

In [ ]:
df =pd.read_csv('/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv')
test =pd.read_csv('/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv')

In [ ]:
df.drop(df.columns[df.columns.str.contains('^h1')], axis=1, inplace=True)

In [ ]:
df.drop(['hospital_id'], axis=1, inplace=True)

In [ ]:
df.drop(df.columns[df.columns.str.contains('^Un')], axis=1, inplace=True)

In [ ]:
df.drop(df.columns[df.columns.str.contains('invasive')], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('noninvasive')], axis=1, inplace=True)

In [ ]:
cols = df.columns[df.isnull().mean()>0.7]

In [ ]:
print(cols)

In [ ]:
len(cols)

In [ ]:
df.drop(cols, axis=1, inplace=True)

In [ ]:
df.info()
df.value_counts()

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
dfCorr = df.corr()

In [ ]:
corrlist = df.corr().unstack().sort_values().drop_duplicates()

In [ ]:
print(corrlist)

In [ ]:
kot = dfCorr[dfCorr>=0.99]

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(kot,cmap="Greens")

In [ ]:
c = df.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")


In [ ]:
print(so)

In [ ]:
print([c for c in df.columns if 7<df[c].nunique()<800])

In [ ]:
print([c for c in df.columns if (1<df[c].nunique()) & (df[c].dtype != np.number)& (df[c].dtype != int) ])

In [ ]:
categorical_cols =  ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
display(df[categorical_cols].isna().sum())

In [ ]:
df[categorical_cols] = df[categorical_cols].fillna("")

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
display(df[categorical_cols].isna().sum())

In [ ]:
#for col in categorical_cols:
#     df[col] = pd.to_numeric(df[col], errors='coerce')
     

In [ ]:
#for col in categorical_cols:
#    df[col]=df[col].fillna(df[col].mode(),inplace=True)

In [ ]:
 df['ethnicity'] = pd.to_numeric(df['ethnicity'],errors='coerce')

In [ ]:
df['gender'] = pd.to_numeric(df['ethnicity'],errors='coerce')
df['hospital_admit_source'] = pd.to_numeric(df['hospital_admit_source'],errors='coerce')
df['icu_admit_source'] = pd.to_numeric(df['icu_admit_source'],errors='coerce')
df['icu_stay_type'] = pd.to_numeric(df['icu_stay_type'],errors='coerce')
df['icu_type'] = pd.to_numeric(df['icu_type'],errors='coerce')

In [ ]:
df.dtypes

In [ ]:
numerical_Columns=[]
for i in df.columns:
    if df[i].dtype=='int32' or df[i].dtype=='float32' or  df[i].dtype=='int64' or df[i].dtype=='float64':
        numerical_Columns.append(i)

In [ ]:
df.dtypes

In [ ]:
df[numerical_Columns] = df[numerical_Columns].fillna(0)

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
df['ethnicity']=df['ethnicity'].fillna(0)

In [ ]:
df.isna().sum()

In [ ]:
df['gender']=df['gender'].fillna(0)
df['hospital_admit_source']=df['hospital_admit_source'].fillna(0)
df['icu_admit_source']=df['icu_admit_source'].fillna(0)
df['icu_stay_type']=df['icu_stay_type'].fillna(0)
df['icu_type']=df['icu_type'].fillna(0)


In [ ]:
df.isna().sum()

In [ ]:
X_train = df.drop([TARGET_COL],axis=1)
y_train = df[TARGET_COL]

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import LogisticRegression


In [ ]:
from sklearn.linear_model import (LinearRegression, Ridge, Lasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# create pipeline
model = LogisticRegression()
#rfe = RFECV(estimator=DecisionTreeClassifier())
#model = DecisionTreeClassifier()
#pipeline = Pipeline(steps=[('s',rfe),('m',model)])

In [ ]:
rfe = RFE(model, 20)
rfe = rfe.fit(X_train, y_train)

In [ ]:
# summarize the selection of the attributes
#print(rfe.support_)
#print(rfe.ranking_)

In [ ]:
# Feature Importance
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
# fit an Extra Trees model to the data
model = ExtraTreesClassifier()


In [ ]:
model.fit(X_train, y_train)
# display the relative importance of each attribute
print(model.feature_importances_)

In [ ]:
importance = model.feature_importances_

In [ ]:
(pd.Series(model.feature_importances_, index=X_train.columns)
   .nlargest(20)
   .plot(kind='barh')) 

In [ ]:
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
from matplotlib import pyplot
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
n_scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
print([c for c in df.columns if 7<df[c].nunique()<800])

In [ ]:
print([c for c in df.columns if (1<df[c].nunique()) & (df[c].dtype != np.number)& (df[c].dtype != int) ])

In [ ]:
categorical_cols =  ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']

In [ ]:
display(df[categorical_cols].dtypes)
display(df[categorical_cols].tail(3))
display(df[categorical_cols].isna().sum())

In [ ]:
df[categorical_cols] = df[categorical_cols].fillna("")

# same transformation for test data
test[categorical_cols] = test[categorical_cols].fillna("")

df[categorical_cols].isna().sum()

In [ ]:
for col in categorical_cols:
     df[col].fillna(df[col].mode()[0],inplace=True)

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
#df['apache_2_diagnosis'] = pd.to_numeric(df['apache_2_diagnosis'], errors='coerce')

In [ ]:
#df['apache_2_diagnosis'] = df['apache_2_diagnosis'].fillna(0)

In [ ]:
#df.apache_3j_diagnosis.unique()

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
#df['apache_3j_diagnosis']=df['apache_3j_diagnosis'].astype(int)#

In [ ]:
#display(df['apache_3j_diagnosis'].dtypes)

In [ ]:
#display(df[categorical_cols].dtypes)#

In [ ]:
#df['apache_2_diagnosis']=df['apache_2_diagnosis'].astype(int)

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
display(df[categorical_cols].tail(3))

In [ ]:
display(df[categorical_cols].isna().sum())

In [ ]:
display(df[categorical_cols].isna().sum())

In [ ]:
df[categorical_cols] = df[categorical_cols].fillna("")

In [ ]:
test[categorical_cols] = test[categorical_cols].fillna("")
df[categorical_cols].isna().sum()
test[categorical_cols].isna().sum()

In [ ]:
X_train = df.drop([TARGET_COL],axis=1)
y_train = df[TARGET_COL]

In [ ]:
df[categorical_cols].info()

In [ ]:
df[categorical_cols].dtypes

In [ ]:
## catBoost Pool object
train_pool = Pool(data=X_train,label = y_train,cat_features=categorical_cols)
#                   baseline= X_train[""], ## 
#                   group_id = X_train['hospital_id']
                 

In [ ]:
model_basic = CatBoostClassifier(verbose=False,iterations=50)#,learning_rate=0.1, task_type="GPU",)

In [ ]:
model_basic.fit(train_pool, plot=True,silent=True)

In [ ]:
print(model_basic.get_best_score())

In [ ]:
### hyperparameter tuning example grid for catboost : 
grid = {'learning_rate': [0.04, 0.1],
        'depth': [7, 11],
#         'l2_leaf_reg': [1, 3,9],
#        "iterations": [500],
       "custom_metric":['Logloss', 'AUC']}

In [ ]:
model = CatBoostClassifier()

In [ ]:
## can also do randomized search - more efficient typically, especially for large search space - `randomized_search`
grid_search_result = model.grid_search(grid, 
                                       train_pool,
                                       plot=True,
                                       refit = True, #  refit best model on all data
                                      partition_random_seed=42)

In [ ]:
print(model.get_best_score())

In [ ]:
print("best model params: \n",grid_search_result["params"])

In [ ]:
feature_importances = model.get_feature_importance(train_pool)

In [ ]:
feature_names = X_train.columns

In [ ]:
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    if score > 0.05:
        print('{0}: {1:.2f}'.format(name, score))

In [ ]:
import shap

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(model)

In [ ]:
shap_values = explainer.shap_values(train_pool)

In [ ]:
# visualize the training set predictions
# SHAP plots for all the data is very slow, so we'll only do it for a sample. Taking the head instead of a random sample is dangerous! 
shap.force_plot(explainer.expected_value,shap_values[0,:400], X_train.iloc[0,:400])

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X_train)

In [ ]:
# test[TARGET_COL] = model.predict(test.drop([TARGET_COL],axis=1),prediction_type='Probability')[:,1]#

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_train)

In [ ]:
shap_values = shap.TreeExplainer(model).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
shap.dependence_plot('diabetes_mellitus', shap_values, X_train)